In [ ]:
import soccerdata as sd
from pathlib import PosixPath
import pandas as pd
import numpy as np

In [ ]:
def get_corner_dict(event_dataframe):
    event_dataframe = event_dataframe[event_dataframe["type"] == "CornerAwarded"]
    event_dataframe = event_dataframe[event_dataframe["outcome_type"] == "Successful"]
    # event_dataframe["time_in_seconds"] = (event_dataframe["minute"] * 60) + event_dataframe["second"]
    corner_dict = {"time": event_dataframe["expanded_minute"].values, "team": event_dataframe["team_id"].values}
    return corner_dict

In [ ]:
ws = sd.WhoScored(
    #  "GER-Bundesliga", "GER-Bundesliga2", "ENG-Premier League", "ESP-La Liga", "FRA-Ligue 1", "ITA-Serie A"
    leagues=["GER-Bundesliga2"],
    seasons=[21],
    no_cache=False,
    no_store=False,
    data_dir=PosixPath("/home/morten/Develop/Open-Data/soccerdata"),
    path_to_browser="/usr/bin/chromium",
    headless=False,
)

In [ ]:
schedule = ws.read_schedule()
schedule.head()

In [ ]:
events = ws.read_events(match_id=1557679)

In [ ]:
events["type"].unique()

In [ ]:
event_end = events.loc[(events["type"] == "End")]
END_OF_GAME = event_end[(event_end["period"] == "SecondHalf") &  (event_end["team_id"] == 39)]["expanded_minute"].values[0]
event_end

In [ ]:
event_dataframe = events.loc[(events["type"] == "SubstitutionOff")]
event_dataframe

In [ ]:
loader = ws.read_events(match_id=1557679, output_fmt='loader')

In [ ]:
def get_player_minutes_dict(loader_players_df, events):
    loader_players_df = loader_players_df[loader_players_df["is_starter"] == True]
    players = np.swapaxes([loader_players_df["player_id"].values, loader_players_df["team_id"].values,
               [0 for _ in range(len(loader_players_df["player_id"].values))], [-1 for _ in range(len(loader_players_df["player_id"].values))]], 0, 1)
    sub_dataframe = events.loc[(events["type"] == "SubstitutionOn") | (events["type"] == "SubstitutionOff")]
    on_dataframe = sub_dataframe.loc[(sub_dataframe["type"] == "SubstitutionOn")].copy()
    # on_dataframe["time_in_seconds"] = (on_dataframe["minute"] * 60) + on_dataframe["second"]
    on_players = np.swapaxes([on_dataframe["player_id"].values.astype(int), on_dataframe["team_id"].values,
               on_dataframe["expanded_minute"].values, [-1 for _ in range(len(on_dataframe["player_id"].values))]], 0, 1)
    players_dict = {}
    for starter in [*players,*on_players]:
        players_dict[starter[0]] = {}
        players_dict[starter[0]]["team_id"] = starter[1]
        players_dict[starter[0]]["on"] = starter[2]
        players_dict[starter[0]]["off"] = starter[3]

    off_dataframe = sub_dataframe.loc[(sub_dataframe["type"] == "SubstitutionOff")].copy()
    # off_dataframe["time_in_seconds"] = (off_dataframe["minute"] * 60) + off_dataframe["second"]
    off_players = np.swapaxes([off_dataframe["player_id"].values.astype(int), off_dataframe["expanded_minute"].values], 0, 1)
    for sub_off in off_players:
        players_dict[sub_off[0]]["off"] = sub_off[1]

    return players_dict

In [ ]:
loader_players_df = loader.players(1557679)

In [ ]:
get_corner_dict(events)

In [ ]:
get_player_minutes_dict(loader_players_df, events)

In [ ]:
def get_corners_per_player(corner_dict, player_dict):
    player_corner = {}
    for player_id in player_dict:
        minutes = END_OF_GAME if ((player_dict[player_id]["off"] == -1) and (player_dict[player_id]["on"] == 0)) else (int(END_OF_GAME - (player_dict[player_id]["on"]))) if (player_dict[player_id]["off"] == -1) and (player_dict[player_id]["on"] != 0 ) else (int((player_dict[player_id]["off"] - player_dict[player_id]["on"])))
        player_corner[player_id] = {"team_id": player_dict[player_id]["team_id"], "for" : 0, "against": 0, "minutes": minutes}
        index_on = next(x[0] for x in enumerate(corner_dict["time"]) if x[1] > player_dict[player_id]["on"])
        index_off = len(corner_dict["time"]) if (player_dict[player_id]["off"] == -1) else next(x[0] for x in enumerate(corner_dict["time"]) if x[1] > player_dict[player_id]["off"]) 
        for index in range(index_on, index_off):
            if corner_dict["team"][index] == player_corner[player_id]["team_id"]:
                player_corner[player_id]["for"] += 1
            else:
                player_corner[player_id]["against"] += 1

    return player_corner

In [ ]:
cpp = get_corners_per_player(get_corner_dict(events), get_player_minutes_dict(loader_players_df, events))

In [ ]:
elo_player = {}
for player in cpp:
    # create test elo dict
    elo_player[player] = 1000

In [ ]:
average_corners_home = [4, 6]
average_corners_away = [3, 6]

In [ ]:
cpp

In [ ]:
def corner_game_off(cpp, p_id, home):
    max_average_corner = max(average_corners_home) if home else max(average_corners_away)
    min_average_corner = min(average_corners_home) if home else min(average_corners_away)
    corners_per_90 = (cpp[p_id]["for"] / cpp[p_id]["minutes"]) * 90
    if corners_per_90 < min_average_corner:
        return 0
    elif corners_per_90 > max_average_corner:
        return 1
    else:
        return 0.5

def corner_game_def(cpp, p_id, home):
    max_average_corner = max(average_corners_home) if home else max(average_corners_away)
    min_average_corner = min(average_corners_home) if home else min(average_corners_away)
    corners_per_90 = (cpp[p_id]["for"] / cpp[p_id]["minutes"]) * 90
    if corners_per_90 > min_average_corner:
        return 0
    elif corners_per_90 < max_average_corner:
        return 1
    else:
        return 0.5

In [ ]:
def update_elo(r_a, r_b, corner_game, K=35):
    R_a = pow(10, (r_a / 400))
    R_b = pow(10, (r_b / 400))
    E_a = R_a / (R_a + R_b)
    E_b = R_b / (R_a + R_b)
    S_a = corner_game # define result
    S_b = 1 - S_a
    r_strich_a = r_a + K * (S_a - E_a)
    r_strich_b = r_b + K * (S_b - E_b)

    print(r_strich_a, r_strich_b)

In [ ]:
update_elo(1500, 1000, 0)

In [ ]:
# create dataframe 
class GameTimeline:
    def __init__(self, player_min_dict) -> None:
        timelines = []
        for player in player_min_dict:
            # get player elo
            player_elo = np.random.randint(600, 1500) # fill with real elo value
            
            player_timeline = np.empty(END_OF_GAME + 3) # + 2 for id and team id, +1 for index of last minute
            player_timeline[:] = np.nan
            player_timeline[0] = player
            player_timeline[1] = player_min_dict[player]["team_id"]
            player_on = player_min_dict[player]["on"] + 2
            player_off = (player_min_dict[player]["off"] + 2) if player_min_dict[player]["off"] != -1 else END_OF_GAME + 3
            player_timeline[player_on : player_off] = player_elo
            timelines.append(player_timeline)
        
        self.game_timeline = pd.DataFrame(timelines, columns=["id", "team_id", *np.arange(END_OF_GAME + 1).astype(str)])

    def get_timeline(self):
        return self.game_timeline

# {13765: {'team_id': 39, 'on': 0, 'off': -1},
#  393355: {'team_id': 39, 'on': 0, 'off': -1},
#  411279: {'team_id': 39, 'on': 0, 'off': 59},
#  82628: {'team_id': 39, 'on': 0, 'off': -1},
#  383806: {'team_id': 39, 'on': 0, 'off': -1},
#  301027: {'team_id': 39, 'on': 0, 'off': -1},
#  70227: {'team_id': 39, 'on': 0, 'off': 31},
#  83804: {'team_id': 39, 'on': 0, 'off': 76},
#  70162: {'team_id': 39, 'on': 0, 'off': -1},
#  361419: {'team_id': 39, 'on': 0, 'off': -1},
#  30464: {'team_id': 39, 'on': 0, 'off': -1},
#  132896: {'team_id': 38, 'on': 0, 'off': -1},
#  132011: {'team_id': 38, 'on': 0, 'off': -1},
#  361253: {'team_id': 38, 'on': 0, 'off': -1},
#  129751: {'team_id': 38, 'on': 0, 'off': -1},
#  140849: {'team_id': 38, 'on': 0, 'off': -1},
#  243758: {'team_id': 38, 'on': 0, 'off': 74},
#  121975: {'team_id': 38, 'on': 0, 'off': -1},
#  343436: {'team_id': 38, 'on': 0, 'off': 70},
#  329689: {'team_id': 38, 'on': 0, 'off': -1},
#  347773: {'team_id': 38, 'on': 0, 'off': 81},
#  136707: {'team_id': 38, 'on': 0, 'off': 81},
#  415639: {'team_id': 39, 'on': 31, 'off': -1},
#  323169: {'team_id': 39, 'on': 59, 'off': -1},
#  280720: {'team_id': 38, 'on': 70, 'off': -1},
#  419916: {'team_id': 38, 'on': 74, 'off': -1},
#  419915: {'team_id': 39, 'on': 76, 'off': -1},
#  392643: {'team_id': 38, 'on': 81, 'off': -1},
#  90966: {'team_id': 38, 'on': 81, 'off': -1}}

In [ ]:
gt = GameTimeline(get_player_minutes_dict(loader_players_df, events))

In [ ]:
timeline = gt.get_timeline()
timeline

In [ ]:
timeline[["29","30", "31", "32"]]